<a href="https://colab.research.google.com/github/RiccardoRubini93/SWA_DQ/blob/main/Upload_data_to_BQ_latest_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.api_core import page_iterator
from google.cloud import storage
import os
import pandas as pd 
from google.cloud import bigquery


In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
def convert_datetime_to_string(data):
  return data.strftime('%Y-%d-%m | %H:%M:%S')

In [ ]:
#next step is to add the extra columns

def bucket_metadata(bucket_name,project_id):
    """Prints out a bucket's metadata."""
  
    storage_client = storage.Client(project_id)
    bucket = storage_client.get_bucket(bucket_name)

    #print(f"ID: {bucket.id}")
    #print(f"Name: {bucket.name}")
    #print(f"Storage Class: {bucket.storage_class}")
    #print(f"Location: {bucket.location}")
    #print(f"Location Type: {bucket.location_type}")

    data                  = {}
    data['bucket_id']     = bucket.id
    data['bucket_name']          = bucket.name
    data['bucket_storage_class'] = bucket.storage_class
    data['bucket_location']      = bucket.location
    data['bucket_location_type'] = bucket.location_type
    #json_data = json.dumps(data)

    return data

def file_path_policy_respected_check(file_path):
    #function that checks whether the path policy of a certain bucket is respected or not
    #each path should be in the following format 
    path_chuncks = file_path.split('/')[2:-1]
    print(path_chuncks)
    if len(path_chuncks) == 5 : policy_respected = True
    else : policy_respected = False

    return policy_respected

def _item_to_value(iterator, item):
    return item

def list_directories(bucket_name, prefix):

    if prefix and not prefix.endswith('/'):
        prefix += '/'

    extra_params = {
        "projection": "noAcl",
        "prefix": prefix,
        "delimiter": '/'
    }

    gcs = storage.Client()

    path = "/b/" + bucket_name + "/o"

    iterator = page_iterator.HTTPIterator(
        client=gcs,
        api_request=gcs._connection.api_request,
        path=path,
        items_key='prefixes',
        item_to_value=_item_to_value,
        extra_params=extra_params,
    )

    return [x for x in iterator]

In [ ]:
#definitions of helpers functions

def file_path_pipeline_code_distinct_versions_check(input_path):

  bucket = input_path.split('/')[2]
  prefix  = input_path.split(bucket)[1][1:-4]

  #build a list containing path and corresponding sizes

  versions_sizes = []

  for directory in list_directories(bucket,prefix):

    path = "gs://"+ bucket + "/" +  directory 
    print(path)
    size = os.popen("gsutil du -s " + str(path)).read().split()
    print("Size : " + str(size))

    versions_sizes.append(size)
  return versions_sizes

def get_file_number_in_landing_zone_path(input_path):

    project_id = 'advanced-analytics-278408'
    client = storage.Client(project_id)

    bucket = input_path.split('/')[2]
    prefix  = input_path.split(bucket)[1][1:-1]

    count = 0

    for blob in client.list_blobs(bucket,prefix=prefix):
      count +=1

    print("Landing zone path : " + str(input_path) )
    print("File number : " + str(count))

    return count

def get_file_size_in_landing_zone_path(input_path):

    bucket = input_path.split('/')[2]
    print(bucket)
    prefix  = input_path.split(bucket)[1][1:-4]
    print(prefix)

    # corresponding sizes

    path = "gs://"+ bucket + "/" +  prefix 
    print(path)
    size = os.popen("gsutil du -s " + str(path)).read().split()[0]
    print("Size : " + str(size))
  
    return round(int(size) / 1024 ** 3, 2)

def get_oldest_and_newest_file(input_path,project_id):

  client = storage.Client(project_id)

  bucket_name = input_path.split('/')[2]
  prefix =      input_path.split(bucket)[1][1:-1]

  #generate a list of blobs

  blobs = [(blob, blob.updated) for blob in client.list_blobs(bucket_name,prefix = prefix,)]

  #sort blobs by update date
  newest  = sorted(blobs, key=lambda tup: tup[1])[-1] #ok funziona
  oldest  = sorted(blobs, key=lambda tup: tup[1])[0]

  return oldest,newest

def is_file_policy_respected(input_path,project_id):

  client = storage.Client(project_id)

  name_is_conformal = True

  print("Check that the policy of the file is respected")
  print("Bucket : " + str(bucket))
  print("Prefix : " + str(prefix))

  for blob in client.list_blobs(bucket,prefix=prefix):

    if len((blob.name.split('/')[-1]).split('__')) != 4 : 
      print("Found not conformal file")
      print(blob.name)
      name_is_conformal = False
      break
  
  return name_is_conformal

def extract_file_info(input_path,project_id):

  client = storage.Client(project_id)

  bucket_name = input_path.split('/')[2]
  prefix =      input_path.split(bucket)[1][1:-1]

  out = {}

  cnt = 0

  for blob in client.list_blobs(bucket,prefix=prefix):
    
    out['file_name_pipeline_code_info'] = blob.name.split('/')[2]
    out['file_name_pipeline_name_info'] = blob.name.split('/')[1]
    try: out['file_name_pipeline_windowframe_info'] = blob.name.split('/')[5].split('__')[2]
    except: out['file_name_pipeline_windowframe_info'] = 'Missing File'

    if cnt > 0 : break

    cnt = +1

  return out

In [ ]:
#generate full list of days for the period covered 

from datetime import datetime, date, timedelta

def generate_list(period_covered,load_time_frame):

  year_s = int(period_covered[0][0:4])
  year_e = int(period_covered[1][0:4])

  month_s = int(period_covered[0][4:6])
  month_e = int(period_covered[1][4:6])

  if load_time_frame == 'dd':

    day_s = int(period_covered[0][6:8])
    day_e = int(period_covered[1][6:8])
    freq = 'd'

  elif load_time_frame == 'mm':

    day_s =  1
    day_e =  1
    freq = 'm'

  start = date(year_s, month_s, day_s)  
  end   = date(year_e, month_e, day_e) 
  
  time_list = []

  if load_time_frame == 'mm':
    
    for item in pd.date_range(start,end,freq=freq).strftime('%Y%m%d').to_list():

      time_list.append(item[0:6])
  
  elif load_time_frame == 'dd':

    time_list = pd.date_range(start,end,freq=freq).strftime('%Y%m%d').to_list() 

  
  return time_list

#select the oldest and newest file -> from filename
def get_file_name_pipeline_history_min_and_max(project_id,file_path):

  client = storage.Client(project_id)

  bucket_name = file_path.split('/')[2]
  prefix      = file_path.split(bucket)[1][1:-1]

  print(bucket_name)
  print(prefix)

  #create a list of date where we pick the oldest and the newest
  blobs = [[blob.name.split('/')[-1].split('__')[-1]] for blob in client.list_blobs(
  bucket_name,
  prefix = prefix)]

  #extract the max and min date

  newest = sorted(blobs, key=lambda tup: tup[0])[-1] 
  oldest = sorted(blobs, key=lambda tup: tup[0])[0]

  return oldest,newest,blobs

def check_missing_days(list1,list2):
  #function that checks if all days are present 
  #list1 contains the set that needs to be checked
  #list2 contains the whole list

  missings = []

  for item in list2:

    if item in list1:
      pass
    else:
      missings.append(item)

  return missings

In [ ]:
from datetime import timezone

def get_warning (path, freq_agg):
  today = datetime.datetime.now(timezone.utc)

  bucket_name = path.split('/')[2]
  prefix  = path.split(bucket_name)[1][1:-1]

  storage_client = storage.Client()
  bucket = storage_client.bucket(bucket_name)

  l = []
  l2 = []

  for directory in list_directories(bucket_name,prefix):
    #print(directory)
    l.append(directory)

  blob_last_directory=max(l)
  print(blob_last_directory)

  files = bucket.list_blobs()    
  fileList = [file.name for file in files if blob_last_directory in file.name]

  for file in fileList:
    blob = bucket.get_blob(file)
    l2.append(blob.updated)
    #print(blob.updated)
  
  blob_last_mod=max(l2)
  print(blob_last_mod)

  if freq_agg=='dd':
    if (today - blob_last_mod).days < 1:
      return 'green'
    elif (today - blob_last_mod).days >= 1 and (today - blob_last_mod).days <= 3:
      return 'yellow'
    else:
      return 'red'

  elif freq_agg=='ww':
    if (today - blob_last_mod).days < 7:
      return 'green'
    elif (today - blob_last_mod).days >= 7 and (today - blob_last_mod).days <= 10:
      return 'yellow'
    else:
      return 'red'
  else:
    if (today - blob_last_mod).days < 30:
      return 'green'
    elif (today - blob_last_mod).days >= 30 and (today - blob_last_mod).days <= 45:
      return 'yellow'
    else:
      return 'red'


In [ ]:
project_id = 'advanced-analytics-278408'
client = bigquery.Client(project=project_id)

query = '''
SELECT
*
FROM `advanced-analytics-278408.data__1st_layer.9891__monitoring_information_and_params`
WHERE is_to_be_monitored = true
and landing_zone = 'GCS';
'''

df = client.query(query).to_dataframe()
df.columns

Index(['project_id', 'dataset_id', 'table_id', 'complete_table_id',
       'table_code', 'is_to_be_monitored', 'current_development_status',
       'current_development_status_id', 'source_system_id',
       'source_system_name', 'source_system_ownership_type', 'extraction_tool',
       'extraction_script', 'extraction_is_scheduled',
       'extraction_schedule_pattern', 'landing_zone', 'landing_zone_path',
       'landing_zone_form', 'loading_tool', 'load_script', 'load_is_scheduled',
       'load_schedule_pattern', 'load_form', 'is_dx_framework_applied',
       'id_field', 'data_loop_field', 'data_loop_granularity',
       'overall_documentation_URL', 'to_do_pending', 'notes'],
      dtype='object')

In [ ]:
df.loc[0,'My_new_col'] = 'new_inserted_value'

In [ ]:
query_2 = '''
SELECT
*
FROM `advanced-analytics-278408.user__rubini_riccardo.9898__pipelines_target_landing_zone_metadata`;
'''

df_9898 = client.query(query_2).to_dataframe()

In [ ]:
df_9898.dtypes

analysis_date                                      object
analysis_date_time                                 object
project_id                                         object
dataset_id                                         object
table_id                                           object
complete_table_id                                  object
table_code                                         object
is_to_be_monitored                                 object
current_development_status                         object
current_development_status_id                      object
source_system_id                                   object
source_system_name                                 object
source_system_ownership_type                       object
extraction_tool                                    object
extraction_script                                  object
extraction_is_scheduled                            object
extraction_schedule_pattern                        object
landing_zone  

In [28]:
import datetime

#fill the first N column of df_9898 with the first N columns of 9891__monitoring_information_and_params
#define a list of columns to fill 

columns_list = [
      'project_id', 'dataset_id', 'table_id', 'complete_table_id',
       'table_code', 'is_to_be_monitored', 'current_development_status',
       'current_development_status_id', 'source_system_id',
       'source_system_name', 'source_system_ownership_type', 'extraction_tool',
       'extraction_script', 'extraction_is_scheduled',
       'extraction_schedule_pattern', 'landing_zone', 'landing_zone_path',
       'landing_zone_form', 'loading_tool', 'load_script', 'load_is_scheduled',
       'load_schedule_pattern', 'load_form'
]


#loop over the columns to fill 

for i in range(df.shape[0]):

  #print the filepath analised
  print("Computing row : " + str(i) + " of " + str(df.shape[0]))
  print(df.loc[i,'landing_zone_path'])

  #fill the date

  df_9898.loc[i,'analysis_date'] = datetime.datetime.now().strftime('%Y-%m-%d')
  df_9898.loc[i,'analysis_date_time'] = datetime.datetime.now()

  #add columns from the 9891
  for col in columns_list:

    df_9898.loc[i,col] = df.loc[i,col]

  bucket = df.loc[i,'landing_zone_path'].split('/')[2]
  bucket_meta = bucket_metadata(bucket,project_id)
  
  #add bucket metadata
  for key in bucket_meta:
    df_9898.loc[i,key] = bucket_meta[key] 

  #check that the policy and path info
  df_9898.loc[i,'file_path_policy_respected'] = file_path_policy_respected_check(df_9898.loc[i,'landing_zone_path'])
  df_9898.loc[i,'file_path_source_system'] = df_9898.loc[i,'landing_zone_path'].split('/')[5]
  df_9898.loc[i,'file_path_pipeline_code'] = df_9898.loc[i,'landing_zone_path'].split('/')[4]

  #extract bucket and prefix for a given landing_zone_path
  bucket = df_9898.loc[i,'landing_zone_path'].split('/')[2]
  prefix  = df_9898.loc[i,'landing_zone_path'].split(bucket)[1][1:-4]
  #count the number of v1 v2 v3 
  print(bucket)
  print(prefix)
  print(list_directories(bucket,prefix))

  df_9898.loc[i,'file_path_pipeline_code_distinct_versions'] = len(list_directories(bucket,prefix))

  #compute the size of each versioning of each folder

  df_9898.loc[i,'file_path_pipeline_code_distinct_versions_sizes'] = str(file_path_pipeline_code_distinct_versions_check(df_9898.loc[i,'landing_zone_path']))
  df_9898.loc[i,'file_path_pipeline_code_checked_version'] = 'TO BE INSERTED'

  #compute the number of files in the landing zone path

  df_9898.loc[i,'file_number_in_landing_zone_path'] = get_file_number_in_landing_zone_path(df_9898.loc[i,'landing_zone_path'],)
 
  df_9898.loc[i,'file_size_in_landing_zone_path'] = get_file_size_in_landing_zone_path(df_9898.loc[i,'landing_zone_path'])

  #get oldest and most recent file in folder

  new = get_oldest_and_newest_file(df_9898.loc[i,'landing_zone_path'],project_id)[0][1]
  old = get_oldest_and_newest_file(df_9898.loc[i,'landing_zone_path'],project_id)[1][1]

  df_9898.loc[i,'file_oldest_creation_datetime'] = old
  df_9898.loc[i,'file_newest_creation_datetime'] = new

  #check policy of the file
  df_9898.loc[i,'file_name_policy_respected']  = is_file_policy_respected(df_9898.loc[i,'landing_zone_path'],project_id)

  #extract file infos

  files_info = extract_file_info(df_9898.loc[i,'landing_zone_path'],project_id)

  df_9898.loc[i,'file_name_pipeline_code_info']        =  files_info['file_name_pipeline_code_info']
  df_9898.loc[i,'file_name_pipeline_name_info']        =  files_info['file_name_pipeline_name_info']
  df_9898.loc[i,'file_name_pipeline_windowframe_info'] =  files_info['file_name_pipeline_windowframe_info']

  #get first and latest date from filename

  old,new,list_day = get_file_name_pipeline_history_min_and_max(project_id,df_9898.loc[i,'landing_zone_path'])
  period_covered = [old[0],new[0]]

  df_9898.loc[i,'file_name_pipeline_history_min_info'] = old[0]
  df_9898.loc[i,'file_name_pipeline_history_max_info'] = new[0]
  df_9898.loc[i,'files_period_covered'] = str(period_covered)

  #get the complete list of days
  try : 
    complete_list_days = generate_list(period_covered,df_9898.loc[i,'file_name_pipeline_windowframe_info'])
    missings = check_missing_days(np.asarray(list_day)[:,0],complete_list_days)

    if len(missings) == 0 : df_9898.loc[i,'files_period_missing_between_min_max'] = 'No missing days'
    else: df_9898.loc[i,'files_period_missing_between_min_max'] = str(missings)
  except:
    print("Missing Files, step skipped")
  
  #get_warning_status
  try : df_9898.loc[i,'warning_flag_status'] = get_warning(df_9898.loc[i,'landing_zone_path'],df_9898.loc[i,'file_name_pipeline_windowframe_info'])
  except : df_9898.loc[i,'warning_flag_status'] = '' 





Computing row : 0 of 48
gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0004/v3/
['swarovski-advanced-analytics-data-ingestion-test', 'clean', '00_SAP_P30', '0004', 'v3']
swarovski-advanced-analytics-data-ingestion-test
clean/00_SAP_P30/0004
['clean/00_SAP_P30/0004/v1/', 'clean/00_SAP_P30/0004/v2/', 'clean/00_SAP_P30/0004/v3/']
gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0004/v1/
Size : ['552271279', 'gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0004/v1']
gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0004/v2/
Size : ['84698956', 'gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0004/v2']
gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0004/v3/
Size : ['235949926', 'gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0004/v3']
Landing zone path : gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0004/v3/

KeyboardInterrupt: ignored

In [ ]:
#select the oldest and newest file -> from filename
def get_file_name_pipeline_history_min_and_max(project_id,file_path):

  client = storage.Client(project_id)

  bucket_name = file_path.split('/')[2]
  prefix      = file_path.split(bucket)[1][1:-1]

  print(bucket_name)
  print(prefix)

  #create a list of date where we pick the oldest and the newest
  blobs = [[blob.name.split('/')[-1].split('__')[-1]] for blob in client.list_blobs(
  bucket_name,
  prefix = prefix)]

  #extract the max and min date

  newest = sorted(blobs, key=lambda tup: tup[0])[-1] 
  oldest = sorted(blobs, key=lambda tup: tup[0])[0]

  return oldest,newest,blobs

#get_file_name_pipeline_history_min_and_max(project_id,df_9898.loc[0,'landing_zone_path']) #test daily load

old,new,list_day = get_file_name_pipeline_history_min_and_max(project_id,df_9898.loc[4,'landing_zone_path'])  #test monthly load

#compute file period covered 

period_covered = [old[0],new[0]]
period_covered


KeyError: ignored

In [ ]:
#generate list of days from the period covered

from datetime import datetime, date, timedelta

def generate_list(period_covered,load_time_frame):

  year_s = int(period_covered[0][0:4])
  year_e = int(period_covered[1][0:4])

  month_s = int(period_covered[0][4:6])
  month_e = int(period_covered[1][4:6])

  if load_time_frame == 'dd':

    day_s = int(period_covered[0][6:8])
    day_e = int(period_covered[1][6:8])
    freq = 'd'

  elif load_time_frame == 'mm':

    day_s =  1
    day_e =  1
    freq = 'm'

  start = date(year_s, month_s, day_s)  
  end   = date(year_e, month_e, day_e) 
  
  time_list = []

  if load_time_frame == 'mm':
    
    for item in pd.date_range(start,end,freq=freq).strftime('%Y%m%d').to_list():

      time_list.append(item[0:6])
  
  elif load_time_frame == 'dd':

    time_list = pd.date_range(start,end,freq=freq).strftime('%Y%m%d').to_list() 

  
  return time_list
    

complete_list = generate_list(period_covered,'mm')
complete_list

['201201',
 '201202',
 '201203',
 '201204',
 '201205',
 '201206',
 '201207',
 '201208',
 '201209',
 '201210',
 '201211',
 '201212',
 '201301',
 '201302',
 '201303',
 '201304',
 '201305',
 '201306',
 '201307',
 '201308',
 '201309',
 '201310',
 '201311',
 '201312',
 '201401',
 '201402',
 '201403',
 '201404',
 '201405',
 '201406',
 '201407',
 '201408',
 '201409',
 '201410',
 '201411',
 '201412',
 '201501',
 '201502',
 '201503',
 '201504',
 '201505',
 '201506',
 '201507',
 '201508',
 '201509',
 '201510',
 '201511',
 '201512',
 '201601',
 '201602',
 '201603',
 '201604',
 '201605',
 '201606',
 '201607',
 '201608',
 '201609',
 '201610',
 '201611',
 '201612',
 '201701',
 '201702',
 '201703',
 '201704',
 '201705',
 '201706',
 '201707',
 '201708',
 '201709',
 '201710',
 '201711',
 '201712',
 '201801',
 '201802',
 '201803',
 '201804',
 '201805',
 '201806',
 '201807',
 '201808',
 '201809',
 '201810',
 '201811',
 '201812',
 '201901',
 '201902',
 '201903',
 '201904',
 '201905',
 '201906',
 '201907',

In [ ]:
#check lists 
def check_missing_days(list1,list2):
  #function that checks if all days are present 
  #list1 contains the set that needs to be checked
  #list2 contains the whole list

  missings = []

  for item in list2:

    if item in list1:
      pass
    else:
      missings.append(item)

  return missings



In [ ]:
import numpy as np

for i in range(len(complete_list)):
  print(complete_list[i],list_day[i][0])

201201 201201
201202 201202
201203 201203
201204 201204
201205 201205
201206 201206
201207 201207
201208 201208
201209 201209
201210 201210
201211 201211
201212 201212
201301 201301
201302 201302
201303 201303
201304 201304
201305 201305
201306 201306
201307 201307
201308 201308
201309 201309
201310 201310
201311 201311
201312 201312
201401 201401
201402 201402
201403 201403
201404 201404
201405 201405
201406 201406
201407 201407
201408 201408
201409 201409
201410 201410
201411 201411
201412 201412
201501 201501
201502 201502
201503 201503
201504 201504
201505 201505
201506 201506
201507 201507
201508 201508
201509 201509
201510 201510
201511 201511
201512 201512
201601 201601
201602 201602
201603 201603
201604 201604
201605 201605
201606 201606
201607 201607
201608 201608
201609 201609
201610 201610
201611 201611
201612 201612
201701 201701
201702 201702
201703 201703
201704 201704
201705 201705
201706 201706
201707 201707
201708 201708
201709 201709
201710 201710
201711 201711
201712

In [ ]:
missings = check_missing_days(np.asarray(list_day)[:,0],complete_list)

if len(missings) == 0 : is_list_empty = True
else: is_list_empty = False

In [ ]:
missings
is_list_empty

True

In [27]:
dataset_id = 'user__rubini_riccardo'

dataset_ref = client.dataset(dataset_id)
job_config = bigquery.LoadJobConfig()
job_config.autodetect = False
job_config.write_disposition = "WRITE_TRUNCATE"
#job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
load_job = client.load_table_from_dataframe(df_9898, dataset_ref.table("9898__pipelines_target_landing_zone_metadata"), job_config=job_config, location="EU")    # API request
print("Starting job {}".format(load_job))

Starting job LoadJob<project=advanced-analytics-278408, location=EU, id=97e9ec9d-5eef-44cf-9a61-6210f3748832>


In [25]:
df

,project_id,dataset_id,table_id,complete_table_id,table_code,is_to_be_monitored,current_development_status,current_development_status_id,source_system_id,source_system_name,...,load_is_scheduled,load_schedule_pattern,load_form,is_dx_framework_applied,id_field,data_loop_field,data_loop_granularity,overall_documentation_URL,to_do_pending,notes
0,advanced-analytics-278408,Draft_Data_Model,0004__v3__traffic_transaction_adj,advanced-analytics-278408.Draft_Data_Model.000...,0004,True,Lab Developed,4,00,SAP P30 (BW query),...,False,,BQ-Uncleansed-No-Common-Naming-Type-No-Descrip...,False,DATE,DATE,DAY,missing,,
1,advanced-analytics-278408,Draft_Data_Model,0006__cogs,advanced-analytics-278408.Draft_Data_Model.000...,0006,True,Lab Developed,4,00,SAP P30 (BW query),...,False,,BQ-Uncleansed-No-Common-Naming-Type-No-Descrip...,False,DATE,DATE,DAY,missing,,
2,advanced-analytics-278408,Draft_Data_Model,0019__cogs_csl,advanced-analytics-278408.Draft_Data_Model.001...,0019,True,Lab Developed,4,00,SAP P30 (BW query),...,False,,BQ-Uncleansed-No-Common-Naming-Type-No-Descrip...,False,DATE,DATE,DAY,missing,,
3,advanced-analytics-278408,Draft_Data_Model,0003__v4__store_master,advanced-analytics-278408.Draft_Data_Model.000...,0003,True,Lab Developed,4,00,SAP P30 (BW query),...,False,,BQ-Uncleansed-No-Common-Naming-Type-No-Descrip...,False,CUSTNUM,DATE,DAY,missing,,
4,advanced-analytics-278408,Draft_Data_Model,0015__returns,advanced-analytics-278408.Draft_Data_Model.001...,0015,True,Lab Developed,4,00,SAP P30 (BW query),...,False,,BQ-Uncleansed-No-Common-Naming-Type-No-Descrip...,False,UNIQUE_KEY_RETAIL_TRANSACTION,DATE,DAY,missing,,
5,advanced-analytics-278408,Draft_Data_Model,0014__repairs,advanced-analytics-278408.Draft_Data_Model.001...,0014,True,Lab Developed,4,00,SAP P30 (BW query),...,False,,BQ-Uncleansed-No-Common-Naming-Type-No-Descrip...,False,column_0CRM_OBJ_ID,ZC_ENTRY,DAY,missing,,
6,advanced-analytics-278408,Draft_Data_Model,0005__online_sales_hist,advanced-analytics-278408.Draft_Data_Model.000...,0005,True,Lab Developed,4,00,SAP P30 (BW query),...,False,,BQ-Uncleansed-No-Common-Naming-Type-No-Descrip...,False,UNIQUE_KEY_TRANSACTION,ORDER_DATE,DAY,missing,,
7,advanced-analytics-278408,Draft_Data_Model,0016__hr_learning,advanced-analytics-278408.Draft_Data_Model.001...,0016,True,Lab Developed,4,00,SAP P30 (BW query),...,False,,BQ-Uncleansed-No-Common-Naming-Type-No-Descrip...,False,Period_first_day,Period_first_day,MONTH,missing,,
8,advanced-analytics-278408,Draft_Data_Model,0017__retail_profit_loss,advanced-analytics-278408.Draft_Data_Model.001...,0017,True,Lab Developed,4,00,SAP P30 (BW query),...,False,,BQ-Uncleansed-No-Common-Naming-Type-No-Descrip...,False,Period_first_day,Period_first_day,MONTH,missing,,
9,advanced-analytics-278408,Draft_Data_Model,0018__capex,advanced-analytics-278408.Draft_Data_Model.001...,0018,True,Lab Developed,4,00,SAP P30 (BW query),...,False,,BQ-Uncleansed-No-Common-Naming-Type-No-Descrip...,False,Period_first_day,Period_first_day,MONTH,missing,,
